In [1]:
from init import *
from math import nan

# Vanilla option

In [2]:
Call = 0
Put = 1

In [3]:
model = sdk.Model()
model.TimeStart = 0
model.TimeSteps = 1000
model.NumPaths = 100000
model.Add(sdk.IndependentGaussian())
underlying = model.Add(sdk.GeometricalBrownianMotion(
    start = 130,
    drift = 0.0,
    diffusion = 0.3
))

option = model.Add(sdk.Option(
    underlying = underlying.GetStateNumber(),
    call_put = sdk.Option.Put,
    strike = 140
))


In [4]:
model.evaluations.append(sdk.EvaluationPoint(0,2))
model.json()

'{"TimeStart": 0, "TimeSteps": 1000, "NumPaths": 100000, "updaters": [{"name": "IndependentGaussian", "refs": [], "args": []}, {"name": "GeometricalBrownianMotion", "refs": [], "args": [0.0, 0.3], "start": 130}, {"name": "Option", "refs": [0], "args": [140, 1], "start": 0}], "evaluations": [{"state": 0, "time": 2, "histograms": []}], "RandomSeed": -1, "RunTimeoutSeconds": 1, "MemoryLimitKB": 1}'

In [5]:
results = run (model)
results.df()

,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,GeometricalBrownianMotion,,0,2.0,1000,100000,129.653336,0.180921,57.212109,1.384727
1,Option,,1,2.0,1000,100000,28.088856,0.091132,28.818348,0.637922


## MC option price vs strike

In [6]:
def vanilla_option_price(spot,strike,volatility,expiry,call_put,mc_paths=10000,mc_steps=1000,seed=-1):
    model = sdk.Model()
    model.TimeStart = 0
    model.TimeSteps = mc_steps
    model.NumPaths = mc_paths
    model.RandomSeed = seed

    model.Add(sdk.IndependentGaussian())
    underlying_process = model.Add(sdk.GeometricalBrownianMotion(spot,0,volatility))

    option_process = model.Add(sdk.Updater(
        name = "Option",
        args = [strike,call_put],
        refs = [underlying_process.GetStateNumber()],
        start = 0
    ))
    
    model.evaluations.append(sdk.EvaluationPoint(0,expiry))

#     response = requests.post(f'{server}/model',model.json())
#     assert response.ok
#     results = sdk.EvaluationResults_from_json_string(response.text)
#     results.model = model
    results = run(model)
        
    return results.GetStateEvaluationResult(option_process.GetStateNumber(),0).mean

In [7]:
import plotly.express as px
import numpy as np

In [ ]:
f = lambda x: vanilla_option_price(130,x,0.3,10,Call,mc_paths=1000)
vx = np.linspace(50,250,20)

fig = px.scatter (
    title = f'Call Option price vs strike',
    x = vx,
    y = [f(x) for x in vx]
)
fig.show()

In [ ]:
f = lambda x: vanilla_option_price(130,x,0.3,10,'call',mc_paths=1000,seed=0)
vx = np.linspace(50,250,20)

fig = px.scatter (
    title = f'Call Option price vs strike',
    x = vx,
    y = [f(x) for x in vx]
)
fig.show()

# Add barrier

In [ ]:
def single_barrier_option_price (
    spot = 130,
    strike = 140,
    volatility = 0.3,
    barrier1_level = 150,
    mc_paths = 10000,
    mc_steps = 1000,
    seed = -1
):
    # prepare model
    model = sdk.Model()
    model.TimeStart = 0
    model.TimeSteps = mc_steps
    model.NumPaths = mc_paths
    model.RandomSeed = seed
    model.Add(sdk.IndependentGaussian())
    # Add a basic 'underlying' process
    underlying_process = model.Add(sdk.GeometricalBrownianMotion(
        start = spot,
        drift = 0.0,
        diffusion = volatility,
        title = 'Underlying'
    ))

    put_option_process = model.Add(sdk.Option(
        underlying = underlying_process.GetStateNumber(),
        strike = strike,
        call_put = sdk.Option.Put,
        title = f'Put, K={strike}'
    ))

    call_option_process = model.Add(sdk.Option(
        underlying = underlying_process.GetStateNumber(),
        strike = strike,
        call_put = sdk.Option.Call,
        title = f'Call, K={strike}'
    ))
    
    UpAndOutDigitalBarrier_process = model.Add(sdk.Barrier(
        underlying = underlying_process.GetStateNumber(),
        start = 1,
        level = barrier1_level,
        direction = sdk.Barrier.DirectionUp,
        action = sdk.Barrier.ActionSet,
        value = 0,
        title = f'Digital:UpAndOut level={barrier1_level}'
    ))

    UpAndInDigitalBarrier_process = model.Add(sdk.Barrier(
        underlying = underlying_process.GetStateNumber(),
        start = 0,
        level = barrier1_level,
        direction = sdk.Barrier.DirectionUp,
        action = sdk.Barrier.ActionSet,
        value = 1,
        title = f'Digital:UpAndIn level={barrier1_level}'
    ))
    
    UpAndOutPutBarrier_process = model.Add(sdk.Multiplication(
        refs = [
            put_option_process.GetStateNumber(),
            UpAndOutDigitalBarrier_process.GetStateNumber()
        ],
        factor = 1,
        title = f'UpAndOutPut level={barrier1_level}'
    ))
    
    UpAndInPutBarrier_process = model.Add(sdk.Multiplication(
        refs = [
            put_option_process.GetStateNumber(),
            UpAndInDigitalBarrier_process.GetStateNumber()
        ],
        factor = 1,
        title = f'UpAndInPut level={barrier1_level}'
    ))

    UpAndOutCallBarrier_process = model.Add(sdk.Multiplication(
        refs = [
            call_option_process.GetStateNumber(),
            UpAndOutDigitalBarrier_process.GetStateNumber()
        ],
        factor = 1,
        title = f'UpAndOutCall level={barrier1_level}'
    ))
    
    UpAndInCallBarrier_process = model.Add(sdk.Multiplication(
        refs = [
            call_option_process.GetStateNumber(),
            UpAndInDigitalBarrier_process.GetStateNumber()
        ],
        factor = 1,
        title = f'UpAndInCall level={barrier1_level}'
    ))

    return model

In [ ]:
model_1b = single_barrier_option_price(
    volatility = 0.1,
    spot       = 130,
    strike     = 140,
    mc_steps   = 1000,
    mc_paths   = 10000,
    seed       = 111
)
model_1b.evaluations.append(sdk.EvaluationPoint(0,10))
results_1b = run (model_1b)
results_1b.df()

### Run the same model, but ask to compute the state at several time points

In [ ]:
model_2b = single_barrier_option_price(
    volatility = 0.1,
    spot       = 130,
    strike     = 140,
    mc_steps   = 1000,
    mc_paths   = 10000,
    seed       = 111
)
model_2b.evaluations.append(sdk.EvaluationPoint(0,2))
model_2b.evaluations.append(sdk.EvaluationPoint(0,5))
model_2b.evaluations.append(sdk.EvaluationPoint(0,10))
results_2b = run (model_2b)
results_2b.df()